**Hotdog or not hotdog**

The program imports a dataset named ["Hot Dog - Not Hot Dog"](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/notebooks) from Kagggle and then creates and trains a Deep Neural Network to classify the images either in hotdog or not hotdog category. 

**STEP 1:** Importing dataset from Kaggle

**DO NOT RUN AGAIN**


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

#changing the working directory
%cd /content/gdrive/My Drive/Kaggle

#checking if in correct directory
!pwd

#downloading dataset from Kaggle
!kaggle datasets download -d dansbecker/hot-dog-not-hot-dog

#Verifying the files were downloaded
!ls

In [ ]:
#unzipping and removing zip file
!unzip \*.zip  && rm *.zip

**Step 2:** Normalizing the data

In [265]:
from PIL import Image
import matplotlib.pyplot as plt
from scipy import ndarray
from os.path import join
import glob
import numpy as np

In [ ]:
%cd /content/gdrive/My Drive/Kaggle/train/hot_dog/
file=glob.glob("*.jpg")
size=64,64
num_of_files=0
#resizing images
for img in file:
  current_img= Image.open(img)
  new_img=current_img.resize(size)
  new_img.save(str(img))
  num_of_files+=1

%cd /content/gdrive/My Drive/Kaggle/train/not_hot_dog/
file=glob.glob("*.jpg")
for img in file:
  num_of_files+=1

X_train=np.empty((num_of_files,64,64,3))
Y_train=np.empty((1,num_of_files))

%cd /content/gdrive/My Drive/Kaggle/train/hot_dog/
file=glob.glob("*.jpg")

#creating an array of images
i=0
for img in file:
  data=np.asarray(Image.open(img))
  X_train[i]=data
  Y_train[0][i]=1
  i+=1

In [ ]:
%cd /content/gdrive/My Drive/Kaggle/train/not_hot_dog/
file=glob.glob("*.jpg")

#resizing images
for img in file:
  current_img= Image.open(img)
  new_img=current_img.resize(size)
  new_img.save(str(img))

#creating an array of images
i=num_of_files//2
for img in file:
  data=np.asarray(Image.open(img))
  X_train[i]=data
  Y_train[0][i]=0
  i+=1

X_train=X_train.reshape(X_train.shape[0],-1).T
print(X_train.shape)

In [ ]:
#standardize the data
X_train=X_train/255
print(X_train.shape)
print(Y_train.shape[1])

**Step 2:** Setting up Neural Network

Contains all the function needed for forward and backward propagation for the neural network.

Defining activation functions that will be needed for the neural network.

In [289]:
import numpy as np

def sigmoid(Z):    
    A = 1/(1+np.exp(-Z))
    
    return A

def relu(Z):

    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    return A


def derivative_relu(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

def derivative_sigmoid(dA, cache):
    Z = cache    
    dZ = dA * sigmoid(Z) * (1-sigmoid(Z))
    
    return dZ

Initializing parameters

In [290]:
def initialize_params(layers):

  parameters={}

  for l in range(1,len(layers)):
    parameters["W"+str(l)]=np.random.randn(layers[l],layers[l-1])*0.01
    parameters["b"+str(l)]=np.zeros((layers[l],1))
    print(parameters["W"+str(l)])
  return parameters 

Forward Propagation: The neural network will consist of a series layers with ReLU as activation function followed by a sigmoid function as the activation function for the last layer. 

In [291]:
def linear_forward(W,A,b):

  Z=np.dot(W,A)+b

  assert(Z.shape==(W.shape[0],A.shape[1]))
  cache=(A,W,b)

  return Z,cache

In [292]:
def forward_activation(W,A,b,activation):

  if activation=="sigmoid":
    Z,linear_cache=linear_forward(W,A,b)
    A=sigmoid(Z)
  
  elif activation=="relu":
    Z,linear_cache=linear_forward(W,A,b)
    A=relu(Z)
  
  activation_cache=(linear_cache,Z)

  return A,activation_cache

In [293]:
def forward_propagation(X,parameters):

  #A0 is X
  A=X
  L=len(parameters)//2
  cache=[]

  #looping through layers and applying the linear function and activation function
  for l in range (1,L):
    A,temp_cache=forward_activation(parameters["W"+str(l)],A,parameters["b"+str(l)],"relu")
    cache.append(temp_cache)

  AL,temp_cache=forward_activation(parameters["W"+str(L)],A,parameters["b"+str(L)],"sigmoid")
  cache.append(temp_cache)

  return AL,cache

Calculating cost

In [294]:
def cost(AL,Y):
  
  m=Y.shape[1]

  cost=-np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))/m

  return np.squeeze(cost)

Backward Propagation

In [295]:
def linear_backward(dZ,cache):

  A,W,b=cache
  m=W.shape[1]

  dW=np.dot(dZ,A.T)/m
  db=np.sum(dZ,axis=1,keepdims=True)/m
  dA=np.dot(W.T,dZ)

  assert (dA.shape == A.shape)
  assert (dW.shape == W.shape)
  assert (db.shape == b.shape)

  return dA,dW,db

In [296]:
def activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = derivative_relu(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)

    elif activation == "sigmoid":
        dZ = derivative_sigmoid(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
    
    return dA_prev, dW, db

In [297]:
def backward_propagation(AL,Y,cache):

  gradient={}
  L=len(cache)
  Y=Y.reshape(AL.shape)

  dAL=np.divide(AL-Y, (1 - AL) *  AL)

  #finding the values of dAL-1,dWL and dbL
  current_cache=cache[L-1]
  gradient["dA"+str(L-1)],gradient["dW"+str(L)],gradient["db"+str(L)]=activation_backward(dAL,current_cache,"sigmoid")

  #looping through the layers 0 to L-2 to find the values of dA,dW and db for layers 1 to L-1
  for l in reversed(range(L-1)):
    current_cache=cache[l]
    dA_temp,dW_temp,db_temp=activation_backward(gradient["dA"+str(l+1)],current_cache,"relu")
    gradient["dA"+str(l)]=dA_temp
    gradient["dW"+str(l+1)]=dW_temp
    gradient["db"+str(l+1)]=db_temp
  
  return gradient

Updating parameters

In [298]:
def update_params(parameters,grads,learning_rate):

  for l in range(1,len(parameters)//2+1):
    parameters["W"+str(l)]=parameters["W"+str(l)]-learning_rate*grads["dW"+str(l)]
    parameters["b"+str(l)]=parameters["b"+str(l)]-learning_rate*grads["db"+str(l)]
  
  return parameters

**Step 3:**Build the Neural Network

In [299]:
def neural_network(X, Y,layers, learning_rate, num_iterations):
    costs = []                         
    parameters = initialize_params(layers)

    for i in range(0, num_iterations):
        AL, cache = forward_propagation(X, parameters)
        gradient = backward_propagation(AL, Y, cache)
        parameters = update_params(parameters, gradient, learning_rate)
        if i%100==0:
          costs.append(cost(AL, Y))
          print(AL)
          
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [ ]:
layers=[X_train.shape[0],700,200,50,10,1]
parameters=neural_network(X_train,Y_train,layers,0.12,1000)

In [311]:
def prediction(X,parameters):
  AL, cache = forward_propagation(X, parameters)
  Y=[]

  for i in range(AL.shape[1]):
    if AL[0][i]<=0.5:
      Y.append(0)
    else:
      Y.append(1)
  return Y

In [312]:
predicted_Y=prediction(X_train,parameters)
print(predicted_Y)

[1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 